In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,ZeroPadding3D
from keras.layers import Conv3D,MaxPooling3D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.optimizers import Adam

Using TensorFlow backend.


#### 定义模型

In [2]:
model = Sequential()
# input : 5x5x200x1 1个channel -> (5,5,200,1)
# layers: 20 filters (3,3,3) 只有 3th dimension 进行 same padding
model.add(ZeroPadding3D(padding=(0, 0, 1)))
model.add(Conv3D(20,(3,3,3),activation='relu',input_shape=(5,5,200,1)))
model.add(MaxPooling3D(pool_size=(3,3,1))) # 200

# 1D 卷积 input  1x1x200
model.add(Conv3D(20,(1,1,3),strides=(1,1,2),padding='same',activation='relu'))
#  参数个数为60, 序列成为100
model.add(Conv3D(20,(1,1,3),strides=(1,1,2),padding='same',activation='relu'))
#  参数个数为60, 序列成为50
model.add(Flatten())
#model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='softmax'))


sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
#adam = Adam(lr=1e-4)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#### 相关函数定义

In [3]:
## 降维操作
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array
  
  # 归一化后降维
def pca_scal_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    spectral_array = preprocessing.scale(spectral_array) # 归一化
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array 

# pca后降维
def pca_scal_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    # 归一化操作
    pca_input_iamge = preprocessing.scale(pca_input_iamge)
    ## 降维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array
##### 不降维，进行归一化处理
def scaler(input_image):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    spectral_array = preprocessing.scale(spectral_array)
    return_array = np.vsplit(spectral_array,145)
    return_array = np.array(return_array)
    return return_array

    
#####-------------------------------------------------- 生成 3D Tensor---------------------------########

# 可以提取出高光谱数据的一个channel的数据 145*145
def split_one_channel(A,channel):
    row = A.shape[0]
    d = [0]*row
    for i in range(row):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(row)))
    return e   
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


def recreate_whole_data(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    return whole_mirror_matrix_array


## 分割patch  work out 一个维度的patch
def create_patch_one(A,fil):
    whole_arr = list()
    row = A.shape[0]-fil+1
    column = A.shape[1]-fil+1
    for i in range(row):
        for j in range(column):
            temp_arr = list()
            for z in range(fil):
                p = A[i+z,j:j+fil]
                temp_arr.append(p)
            one_arr = np.vstack(temp_arr)
            whole_arr.append(one_arr)
    return np.array(whole_arr)
    
def create_patch_whole(A,fil,dim):
    temp = [create_patch_one(split_one_channel(A,i),fil) for i in range(1,dim+1)]
    return np.array(temp)


def create_patch_cube(patch,fil,dim):
    temp = list()
    nums = patch[0].shape[0] # the nums of every dim of windows  n - f + 1
    for j in range(nums):
        temp_re = np.vstack([patch[i][j] for i in range(dim)]).reshape(dim,fil,fil)
        temp_re1= temp_re.transpose(2,1,0) # switch 1,3 dim
        temp_re2 = temp_re1.transpose(1,0,2) # switch 1,2 dim
        temp.append(temp_re2)
    patch_cubes = np.array(temp)
    return patch_cubes

def whole_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    whole_sample_lable_array = np.array(sample_lable_list)
    return whole_sample_lable_array

# 提取有用的标签
def useful_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
        if out_labels[i] != 0:
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    sample_lable_array = np.array(sample_lable_list)
    return sample_lable_array

  
# 存储numpy文件
def save_sample_label(sample_lable_array,path='H:\data\sample_lable_array'):
    np.save(path,sample_lable_array)
    
# 载入numpy文件
def load_sample_label(name):
    s = np.load(name)
    return s

#### 数据预处理

In [4]:
DIM = 200
FIL = 5

In [5]:
inp = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
out = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [6]:
return_array_alpha = scaler(inp)

D:\anaconda\envs\keras\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [7]:
return_array_alpha.shape

(145, 145, 200)

In [8]:
return_array_beta = recreate_whole_data(DIM,FIL,return_array_alpha)

In [9]:
return_array_beta.shape

(200, 149, 149)

In [10]:
return_array_beta = return_array_beta.transpose(2,1,0)

In [11]:
return_array_beta.shape

(149, 149, 200)

In [12]:
return_array_beta=return_array_beta.transpose(1,0,2)

In [13]:
return_array_beta.shape

(149, 149, 200)

In [14]:
## ## 将每个 filxfil (5x5)的patch取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(return_array_beta,FIL,DIM)

In [15]:
patch_dim.shape

(200, 21025, 5, 5)

In [16]:
patch_cubes = create_patch_cube(patch_dim,FIL, DIM)

In [17]:
patch_cubes.shape

(21025, 5, 5, 200)

In [18]:
out_labels = out.reshape(145*145,)

In [19]:
sample_lable_array = useful_sample_lable(patch_cubes)

In [20]:
sample_lable_array.shape 

(10249, 2)

In [21]:
sample_lable_array[0][0].shape

(5, 5, 200)

In [22]:
save_sample_label(sample_lable_array,'F:\data\sample_lable_array')

In [23]:
whole_data = load_sample_label('F:\data\sample_lable_array.npy')

In [24]:
whole_data.shape

(10249, 2)

In [25]:
Data = np.array([whole_data[:,0][i]for i in range(whole_data.shape[0])])  ## 取出数据

In [26]:
Lable = sample_lable_array[:,-1] -1 ## 取出标签
lable_ = keras.utils.to_categorical(Lable,num_classes=16)

In [27]:
Data.shape

(10249, 5, 5, 200)

In [28]:
data_train, data_test, lable_train, lable_test = train_test_split(Data,lable_,test_size=0.8) ## 分割训练测试集合

In [33]:
#data_train.shape
data_train = data_train.reshape(2049,5,5,200,1)
#data_train = data_train.reshape(1024,5,5,200)

In [34]:
data_train.shape

(2049, 5, 5, 200, 1)

In [35]:
data_train[0][0].shape

(5, 200, 1)

In [36]:
#data_test.shape
data_test = data_test.reshape(8200, 5, 5, 200,1)
#data_test = data_test.reshape(9225,5,5,200)

In [37]:
data_test.shape

(8200, 5, 5, 200, 1)

In [38]:
lable_train.shape

(2049, 16)

In [39]:
lable_test.shape

(8200, 16)

In [40]:
#### --------------------------------放入云服务器计算-----------------------------------------######
print('Training ------------------------')
model.fit(data_train, lable_train, batch_size=16, epochs=100)
print('Tseting -------------------------')
score = model.evaluate(data_test, lable_test, batch_size=16)
print('\ntest loss: ', score[0])
print('\ntest accuracy', score[1])

Training ------------------------
Epoch 1/100
2049/2049 [==============================] - 27s 13ms/step - loss: 1.5934 - acc: 0.4895
Epoch 2/100
2049/2049 [==============================] - 25s 12ms/step - loss: 1.0518 - acc: 0.6164
Epoch 3/100
2049/2049 [==============================] - 25s 12ms/step - loss: 0.8738 - acc: 0.6906
Epoch 4/100
2049/2049 [==============================] - 25s 12ms/step - loss: 0.6923 - acc: 0.7531
Epoch 5/100
2049/2049 [==============================] - 25s 12ms/step - loss: 0.5793 - acc: 0.7940 3s - loss: 0.5
Epoch 6/100
2049/2049 [==============================] - 25s 12ms/step - loss: 0.4653 - acc: 0.8263
Epoch 7/100
2049/2049 [==============================] - 25s 12ms/step - loss: 0.3882 - acc: 0.8507: 0s - loss: 0.3887 - acc: 0.
Epoch 8/100
2049/2049 [==============================] - 29s 14ms/step - loss: 0.3164 - acc: 0.8848
Epoch 9/100
2049/2049 [==============================] - 31s 15ms/step - loss: 0.3054 - acc: 0.8882
Epoch 10/100
2049/2049

2049/2049 [==============================] - 30s 14ms/step - loss: 9.3893e-05 - acc: 1.0000
Epoch 80/100
2049/2049 [==============================] - 28s 14ms/step - loss: 9.2417e-05 - acc: 1.0000
Epoch 81/100
2049/2049 [==============================] - 28s 14ms/step - loss: 9.0914e-05 - acc: 1.0000
Epoch 82/100
2049/2049 [==============================] - 28s 14ms/step - loss: 8.9174e-05 - acc: 1.0000
Epoch 83/100
2049/2049 [==============================] - 28s 14ms/step - loss: 8.7982e-05 - acc: 1.0000
Epoch 84/100
2049/2049 [==============================] - 28s 14ms/step - loss: 8.6863e-05 - acc: 1.0000
Epoch 85/100
2049/2049 [==============================] - 30s 15ms/step - loss: 8.5231e-05 - acc: 1.0000
Epoch 86/100
2049/2049 [==============================] - 29s 14ms/step - loss: 8.3898e-05 - acc: 1.0000
Epoch 87/100
2049/2049 [==============================] - 29s 14ms/step - loss: 8.2485e-05 - acc: 1.0000
Epoch 88/100
2049/2049 [==============================] - 29s 14ms/s

In [41]:
from keras.models import load_model

In [45]:
model.save('dcnn_model.h5')